# Model Training
Uses TensorFlow and Keras to train a model from a dataset, exports model. Intended to be run once, and model is intended to be exported.
Source: keras documentation, https://keras.io/examples/vision/handwriting_recognition/

In [1]:
!wget -q https://github.com/sayakpaul/Handwriting-Recognizer-in-Keras/releases/download/v1.0.0/IAM_Words.zip
!unzip -qq IAM_Words.zip
!
!mkdir data
!mkdir data/words
!tar -xf IAM_Words/words.tgz -C data/words
!mv IAM_Words/words.txt data

mkdir: data: File exists
mkdir: data/words: File exists


# Imports + Seed
## Imports:
- StringLookup from keras. Part of preprocessing layer, maps characters to integers and vice-versa
- matplotlib. Graphing
- tensorflow. ML, includes keras
- numpy. Number processing stuff.
- os. Operating system stuff.

## Note about seed
Numpy has its own specific algorithm for its random number generator. That algorithm has an input, and if the exact same input is given, the exact same set of random numbers will appear in the output. (source: https://keras.io/examples/vision/handwriting_recognition/)
The documentation includes two lines that set the seed value. I have no idea why.

In [ ]:
from tensorflow.keras.layers import StringLookup

import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import os

# see note above
np.random.seed(42)
tf.random.set_seed(42)